# **Convolutional Neural Network**

## **Libraries**

In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import numpy as np
import torch
import time

In [2]:
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## **Settings**

In [3]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [4]:
random_seed = 1
learning_rate = 0.05
num_epochs = 10
batch_size = 128

In [5]:
num_classes = 10

## **MNIST dataset**

In [7]:
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(),download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor())

In [8]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


## **Building the model**

In [10]:
class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()        

        self.conv_1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=1) 
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
        self.conv_2 = torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=1)        
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0) 

        self.linear_1 = torch.nn.Linear(7*7*16, num_classes)

        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
                m.weight.data.normal_(0.0, 0.01)
                m.bias.data.zero_()
                if m.bias is not None:
                    m.bias.detach().zero_()
        
        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)

        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)
        
        logits = self.linear_1(out.view(-1, 7*7*16))
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [11]:
torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)

In [12]:
model = model.to(device)

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

## **Training phase**

In [14]:
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = features.to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [16]:
start_time = time.time()    
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        optimizer.step()
        
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))
    
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/469 | Cost: 2.3026
Epoch: 001/010 | Batch 050/469 | Cost: 2.2991
Epoch: 001/010 | Batch 100/469 | Cost: 2.2996
Epoch: 001/010 | Batch 150/469 | Cost: 2.2982
Epoch: 001/010 | Batch 200/469 | Cost: 2.3028
Epoch: 001/010 | Batch 250/469 | Cost: 2.2980
Epoch: 001/010 | Batch 300/469 | Cost: 2.3024
Epoch: 001/010 | Batch 350/469 | Cost: 2.3009
Epoch: 001/010 | Batch 400/469 | Cost: 2.2967
Epoch: 001/010 | Batch 450/469 | Cost: 2.2740
Epoch: 001/010 training accuracy: 32.29%
Time elapsed: 0.74 min
Epoch: 002/010 | Batch 000/469 | Cost: 2.2205
Epoch: 002/010 | Batch 050/469 | Cost: 1.1182
Epoch: 002/010 | Batch 100/469 | Cost: 0.8915
Epoch: 002/010 | Batch 150/469 | Cost: 0.5937
Epoch: 002/010 | Batch 200/469 | Cost: 0.4591
Epoch: 002/010 | Batch 250/469 | Cost: 0.2613
Epoch: 002/010 | Batch 300/469 | Cost: 0.3219
Epoch: 002/010 | Batch 350/469 | Cost: 0.2371
Epoch: 002/010 | Batch 400/469 | Cost: 0.2065
Epoch: 002/010 | Batch 450/469 | Cost: 0.1443
Epoch: 002/010 t

In [17]:
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Total Training Time: 7.58 min


## **Evaluation**

In [18]:
with torch.set_grad_enabled(False):
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 98.25%
